<a href="https://colab.research.google.com/github/r-gamma/google_colab_playground/blob/main/docker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get -qq install docker.io

In [ ]:
%%bash
docker --version
# docker run hello-world --init

In [ ]:
%%bash
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker network ls
docker images
docker run ubuntu
docker ps

docker inspect --format='{{.State.Status}}' hello-world
docker info
docker pull hello-world

docker build -t hello-world .
docker inspect --format='{{json .State}}'　hello-world

kill $(jobs -p)

In [ ]:
# 以降ちょっとうまくいってない
# https://stackoverflow.com/questions/60235353/docker-is-not-running-on-colab
%%shell
useradd -md /opt/docker docker
apt-get -qq install iproute2 uidmap
sudo -Hu docker SKIP_IPTABLES=1 bash < <(curl -fsSL https://get.docker.com/rootless)

In [ ]:
%%writefile docker-run.sh
#!/usr/bin/env bash
set -e
export DOCKER_SOCK=/opt/docker/.docker/run/docker.sock
export DOCKER_HOST=unix://$DOCKER_SOCK
export PATH=/opt/docker/bin:$PATH
export XDG_RUNTIME_DIR=/opt/docker/.docker/run
/opt/docker/bin/dockerd-rootless.sh --experimental --iptables=false --storage-driver vfs &
for i in $(seq 5); do [ ! -S "$DOCKER_SOCK" ] && sleep 2 || break; done
docker run $@
jobs -p
docker logs
kill $(jobs -p)

In [ ]:
!sudo -Hu docker bash -x docker-run.sh hello-world

In [ ]:
!mount -vt sysfs sysfs /sys -o rw,remount
!mount -vt tmpfs tmpfs /sys/fs/cgroup -o rw,remount

In [ ]:
%%writefile docker-run.sh
#!/usr/bin/env bash
set -e
export DOCKER_SOCK=/opt/docker/.docker/run/docker.sock
export DOCKER_HOST=unix://$DOCKER_SOCK
export PATH=/opt/docker/bin:$PATH
export XDG_RUNTIME_DIR=/opt/docker/.docker/run
rootlesskit --debug --disable-host-loopback --copy-up=/etc --copy-up=/run /opt/docker/bin/dockerd -b none --experimental --iptables=false --storage-driver vfs &
for i in $(seq 5); do [ ! -S "$DOCKER_SOCK" ] && sleep 2 || break; done
docker $@
jobs -p
kill $(jobs -p)

In [ ]:
!sudo -Hu docker bash docker-run.sh run --cap-add SYS_ADMIN hello-world
